In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Load and preprocess the dataset for Vanguard Total Bond Market ETF (BND)
data_path = '/content/drive/My Drive/TimeSeries_Portfolio_Optimization/bond_data.csv'  # Update path to your data file
data = pd.read_csv(data_path, parse_dates=['Date'], index_col='Date')
data = data[['Close']]  # Selecting only the closing price

# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]

# Scale the data for LSTM
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# Define generator parameters
n_input = 30
n_features = 1
generator = TimeseriesGenerator(train_scaled, train_scaled, length=n_input, batch_size=32)

# Build and train the LSTM model
lstm_model = Sequential([
    LSTM(50, activation='relu', input_shape=(n_input, n_features)),
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.fit(generator, epochs=20)

# Save the trained model
model_path = '/content/drive/My Drive/TimeSeries_Forecast_BND/lstm_model_bnd.h5'
lstm_model.save(model_path)

# Load the saved model
# Load the saved model without compiling
lstm_model = load_model(model_path, compile=False)


# Forecast future data
lstm_forecast = []
first_eval_batch = train_scaled[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_forecast.append(lstm_pred)
    current_batch = np.append(current_batch[:, 1:, :], [[lstm_pred]], axis=1)

# Inverse transform to get actual prices
lstm_forecast = scaler.inverse_transform(lstm_forecast).flatten()

# Evaluate and plot the forecast
test_dates = test.index
plt.figure(figsize=(12, 6))
plt.plot(train.index, train['Close'], label='Training Data')
plt.plot(test.index, test['Close'], label='Actual BND Price')
plt.plot(test_dates, lstm_forecast, label='LSTM Forecast')
plt.title('LSTM Model - Vanguard Total Bond Market ETF (BND) Forecast')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()

# Forecast analysis and trend identification
forecast_df = pd.DataFrame({'Forecast': lstm_forecast}, index=test_dates)
forecast_std = np.std(lstm_forecast)
upper_bound = forecast_df['Forecast'] + (1.96 * forecast_std)
lower_bound = forecast_df['Forecast'] - (1.96 * forecast_std)

# Plot forecast with confidence intervals
plt.figure(figsize=(12, 6))
plt.plot(train.index, train['Close'], label='Training Data')
plt.plot(test.index, test['Close'], label='Actual BND Price')
plt.plot(test_dates, lstm_forecast, label='Forecast')
plt.fill_between(test_dates, lower_bound, upper_bound, color='gray', alpha=0.3)
plt.title('LSTM Forecast with Confidence Intervals for BND')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()

# Interpret results
if upper_bound[-1] > forecast_df['Forecast'].iloc[-1]:
    print("Market Opportunity: There is potential for price increases, which may present buying opportunities.")
threshold = 0.05  # Example threshold for high volatility
if forecast_std > threshold:
    print("Market Risk: High volatility could mean increased investment risk, especially if unexpected events impact prices.")


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model without compiling to bypass missing metric/loss errors
model_path = '/content/drive/My Drive/TimeSeries_Portfolio_Optimization/lstm_model.h5'
lstm_model = load_model(model_path, compile=False)

# If needed, recompile with new settings
lstm_model.compile(optimizer='adam', loss='mse')


In [ ]:
# Forecast parameters
forecast_steps = 180  # 6 months of daily data, adjust for 12 months as needed
sequence_length = 60  # Using 60 days of data for each prediction

# Prepare the input sequence
last_sequence = scaled_data[-sequence_length:]
forecast_input = last_sequence.reshape((1, sequence_length, 1))


In [ ]:
# Generate forecast
forecast = []
for _ in range(forecast_steps):
    # Predict the next value
    predicted_price = lstm_model.predict(forecast_input)
    forecast.append(predicted_price[0, 0])
    
    # Update the forecast input sequence for the next step
    forecast_input = np.append(forecast_input[:, 1:, :], [[predicted_price]], axis=1)

# Transform forecast back to original scale
forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))


In [ ]:
# Prepare forecast DataFrame
forecast_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=forecast_steps)
forecast_df = pd.DataFrame(forecast, index=forecast_dates, columns=['Forecast'])

# Plot the results
plt.figure(figsize=(14, 7))
plt.plot(df['Close'], label='Historical Data')
plt.plot(forecast_df['Forecast'], label='Forecasted Data', color='orange')
plt.fill_between(forecast_df.index,
                 forecast_df['Forecast'] * 0.95,  # Lower bound (5% deviation)
                 forecast_df['Forecast'] * 1.05,  # Upper bound (5% deviation)
                 color='gray', alpha=0.2, label='Confidence Interval')
plt.title('Vanguard Total Bond Market ETF (BND) - 6-Month Forecast')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()
